In [ ]:
import math
import random
import plotly.express as px
import pandas as pd 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pickle,random
import plotly.graph_objects as go

def multiply_list(lst):
    out = [None] * (2*len(lst))
    out[::2] = lst
    out[1::2] = lst
    return out


def create_visualization(data_df,user_id,week_num,data_summary_user_current):
   
    data_df=data_df.loc[data_df['Stressor'].notnull()]
    data_df=data_df.loc[data_df['Location'].notnull()]
    user_df=data_df.loc[data_df.user==user_id]
    with open('/home/jupyter/sneupane/MOODS/pickled_files/colors.pickle', 'rb') as handle:
        colors = pickle.load(handle)
    random.shuffle(colors)
    
    if len(user_df)>0:
        location_dict={}
        with open('/home/jupyter/sneupane/MOODS/pickled_files/location_abbr.pickle', 'rb') as handle:
            location_lemma = pickle.load(handle)
        y_modified_labels=location_lemma[user_id]
#         temp_df=user_df.groupby(["Location"],as_index=False)["duration"].sum()
#         temp_df=temp_df.sort_values(["duration"],ascending=False)
#         temp_df["Location_abr"]=temp_df["Location"].map(y_modified_labels)
        user_df_mod=user_df.replace(user_df.groupby('Location').sum().sort_values('duration', ascending=False).index[10:], 'Others').\
            groupby('Location').sum().sort_values(["duration"],ascending=False).reset_index()
        
        Locations=user_df_mod.Location.values.tolist()
        if "Others" in Locations:
            Locations.append(Locations.pop(Locations.index('Others')))
            user_df_mod=user_df_mod.set_index('Location')
            user_df_mod=user_df_mod.loc[Locations].reset_index()
        user_df_mod["Location_abr"]=user_df_mod["Location"].map(y_modified_labels)
        user_df_mod=user_df_mod.round(2)
        
        
        
       

        fig=go.Figure(data=[go.Pie(labels=user_df_mod["Location_abr"], values=user_df_mod["duration"],legendgroup="one",customdata=user_df_mod,
                            hole=.4,textinfo='label+percent',sort=False,direction="clockwise",showlegend=False,
                                insidetextorientation='radial',textposition="outside", hovertemplate=
                                "<b>Location</b>: %{customdata[0][0]}<br>" +
                                "<b>Percentage</b>: %{percent}<br>" +        
                                "<extra></extra>", 
                                   )])
        fig.update_layout(hoverlabel=dict(
                                    bgcolor="white",
                                    font_size=16,
                                    font_family="Rockwell",
                                    bordercolor="Black"
                                ),
                          font=dict(
        family="Courier New, monospace",
        size=12,
        color="Black"
    ))
        fig.update_layout(
                    autosize=True,

                    margin=dict(
                        l=10,
                        r=10,
                        b=10,
                        t=10,
                       
                    ),)
        fig.update_traces(
                  marker=dict(colors=colors, 
                              line=dict(color='#000000', width=2)))
        data_summary_user=data_summary_user_current["Location Prominence"]        
        user_df_this_week=user_df.loc[user_df["week"] == week_num]        
        if len(user_df_this_week) > 0:           
                user_df_this_week_group=user_df_this_week.groupby(["Location"],as_index=False)["duration"].sum().sort_values(["duration"],ascending=False)                
                location_this_week=user_df_this_week_group["Location"].iloc[0]
                data_summary_user=data_summary_user.replace("_a_", location_this_week)            
        else:           
            data_summary_user=data_summary_user.replace("_a_", str("No stressors reported this week"))      
        
        
        data_summary_user=data_summary_user.replace("week_num", str(week_num))
        
        return  "Location Prominence",fig,data_summary_user
    else:
        return "","",""